## **Importing the necessary libraries**

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import os
import csv
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import LearningRateScheduler

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

## Model for male gender

In [3]:
def lr_scheduler(epoch, lr):
    if epoch % 10 == 0 and epoch != 0:
        lr = lr * 0.1
    return lr

# Load the pre-trained VGG model
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Create a functional model
input_layer = vgg_model.input

# # Add additional layers on top of VGG model
# x = Conv2D(64, (3, 3), padding='same', kernel_regularizer=l2(0.001))(vgg_model.output)
# x = BatchNormalization()(x)
# x = Activation('relu')(x)
# x = MaxPooling2D(pool_size=(2, 2))(x)
# x = Dropout(0.25)(x)

# x = Conv2D(512, (3, 3), padding='same', kernel_regularizer=l2(0.001))(x)
# x = BatchNormalization()(x)
# x = Activation('relu')(x)
# x = MaxPooling2D(pool_size=(2, 2))(x)
# x = Dropout(0.25)(x)

# x = Flatten()(x)

# x = Dense(256, kernel_regularizer=l2(0.001))(x)
# x = BatchNormalization()(x)
# x = Activation('relu')(x)
# x = Dropout(0.25)(x)

x = Dense(512, kernel_regularizer=l2(0.001))(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.25)(x)

# Add the first output layer for emotion prediction
emotion_output = Dense(8, activation='softmax', name='emotion_output')(x)

# Add the second output layer for intensity prediction
intensity_output = Dense(2, activation='softmax', name='intensity_output')(x)

# Create the model with multiple outputs
model = Model(inputs=input_layer, outputs=[emotion_output, intensity_output])

# Compile the model
opt = Adam(lr=0.001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

# Learning rate scheduler
lr_schedule = LearningRateScheduler(lr_scheduler)



58889256/58889256 [==============================] - 2s 0us/step


NameError: name 'x' is not defined

In [ ]:
# Load the training CSV file
train_data = pd.read_csv("/kaggle/working/male_train.csv")

# Load the testing CSV file
test_data = pd.read_csv("/kaggle/working/male_test.csv")

# Extract the image paths and labels from the training CSV file
train_image_paths = train_data['path'].tolist()
emotion_labels_train = train_data['emotion'].tolist()
intensity_labels_train = train_data['intensity'].tolist()

# Extract the image paths and labels from the testing CSV file
test_image_paths = test_data['path'].tolist()
emotion_labels_test = test_data['emotion'].tolist()
intensity_labels_test = test_data['intensity'].tolist()

# Convert the labels to categorical format
emotion_labels_train = to_categorical(emotion_labels_train, dtype='uint8')
intensity_labels_train = to_categorical(intensity_labels_train, dtype='uint8')
emotion_labels_test = to_categorical(emotion_labels_test, dtype='uint8')
intensity_labels_test = to_categorical(intensity_labels_test, dtype='uint8')

# Load the training images
train_images = []
for path in train_image_paths:
    train_images.append(np.load(path))

# Load the testing images
test_images = []
for path in test_image_paths:
    test_images.append(np.load(path))

# Train the model
history = model.fit(np.array(train_images), {'emotion_output': emotion_labels_train, 'intensity_output': intensity_labels_train},
                    validation_data=(np.array(test_images), {'emotion_output': emotion_labels_test, 'intensity_output': intensity_labels_test}),
                    epochs=10, batch_size=2)
